# Grid Search Analysis for Alpha, Beta, Delta

This notebook analyzes the results of a coarse parameter sweep over α, β, and δ. Each CSV contains the result of one trial for a specific (α, β, δ) combination.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import re
from glob import glob

# Directory where simulation CSVs are stored
csv_folder = "GridFine"

# Regex to extract alpha, beta, delta from filenames like A2m_B10m_D5m_Data0.csv
pattern = re.compile(r"A(\d+)m_B(\d+)m_D(\d+)m_Data\d+\.csv")

results = {}

for filepath in glob(os.path.join(csv_folder, "*.csv")):
    filename = os.path.basename(filepath)
    match = pattern.match(filename)
    if not match:
        continue

    alpha = int(match.group(1)) / 1000.0
    beta  = int(match.group(2)) / 1000.0
    delta = int(match.group(3)) / 1000.0
    key = (alpha, beta, delta)

    try:
        df = pd.read_csv(filepath, sep=None, engine="python")
        error_y = df.iloc[:, 0]  # Assumes first column is error_y
        cost = np.sum(error_y ** 2)
    except Exception:
        continue

    if key not in results:
        results[key] = []
    results[key].append(cost)


In [2]:
# Create summary DataFrame
data = []
for (alpha, beta, delta), costs in results.items():
    data.append({
        "alpha": alpha,
        "beta": beta,
        "delta": delta,
        "mean_cost": np.mean(costs),
        "std_cost": np.std(costs),
        "n_trials": len(costs)
    })

df_summary = pd.DataFrame(data)
df_summary.sort_values("mean_cost", inplace=True)
df_summary.head()


KeyError: 'mean_cost'

In [ ]:
# Visualize top 10 combinations
top10 = df_summary.head(10)
sns.set(style="whitegrid")
plt.figure(figsize=(10, 6))
sns.barplot(data=top10, x="mean_cost", y=top10.apply(lambda r: f"A{int(r.alpha*1000)}m_B{int(r.beta*1000)}m_D{int(r.delta*1000)}m", axis=1))
plt.xlabel("Mean Cost")
plt.ylabel("Parameter Combination")
plt.title("Top 10 Parameter Sets by Mean Cost")
plt.tight_layout()
plt.show()
